In [4]:
import random
import pygame
import numpy as np

random_seed = 42

Initialization

In [5]:
def create_initiate_state(n: int):
    all_possible_positions = [(i, j) for i in range(n) for j in range(n)]
    sample = random.sample(all_possible_positions, n)
    return set(sample)


In [6]:
def get_matrix_view(positions: set, n: int):
    matrix = np.zeros((n, n), dtype=int)
    for i, j in positions:
        matrix[i][j] = 1
    return matrix

In [7]:
random.seed(random_seed)
queens = create_initiate_state(5)
get_matrix_view(queens, 5)

array([[1, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [1, 0, 0, 1, 0]])

In [8]:
def get_state_neighbors(queen: tuple[int, int], n: int, depth: int, obstacles: set | None = None):
    neighbors = set()
    q_row, q_col = queen
    horizontal_directions = [(0, 1), (0, -1)]
    vertical_directions = [(1, 0), (-1, 0)]
    diag_directions = [(1, 1), (-1, 1), (1, -1), (-1, -1)]
    directions = horizontal_directions + vertical_directions + diag_directions
    for direction in directions:
        d_row, d_col = direction
        c_row, c_col = q_row + d_row, q_col + d_col
        current_depth = 1
        while 0 <= c_row < n and 0 <= c_col < n and current_depth <= depth:
            if obstacles and (c_row, c_col) in obstacles:
                break
            neighbors.add((c_row, c_col))
            c_row, c_col = c_row + d_row, c_col + d_col
            current_depth += 1
    return neighbors

In [9]:
def heuristic(queens: set):
    h = 0
    n = len(queens)
    remaining_queens: set = queens.copy()
    while len(remaining_queens) > 0:
        queen = remaining_queens.pop()
        neighbors = get_state_neighbors(queen, n, depth=n)
        h += len(neighbors & remaining_queens)
    return h


In [10]:
heuristic(queens)

7

In [69]:
def get_all_neighbor_states(queens: set, depth: int):
    n = len(queens)
    neighbors_states = list()
    for queen in queens:
        remaining_queens = queens - {queen}
        neighbors = get_state_neighbors(queen, n, depth, remaining_queens)
        for neighbor in neighbors:
            neighbors_states.append(remaining_queens | {neighbor})
    return neighbors_states

In [70]:
def get_best_state(states: list):
    best_state = states.pop()
    best_h = heuristic(best_state)
    for state in states:
        h = heuristic(state)
        if h < best_h:
            best_h = h
            best_state = state
    return best_state, best_h

In [71]:
def hill_climbing(initial_state, depth: int, side_way_moves: int = 0, ):
    current_state, current_h = initial_state, heuristic(initial_state)
    current_side_way_moves = 0
    while True:
        all_neighbors = get_all_neighbor_states(current_state, depth)
        best_neighbor_state, best_neighbor_h = get_best_state(all_neighbors)
        if best_neighbor_h == current_h:
            if current_side_way_moves < side_way_moves:
                current_side_way_moves += 1
            else:
                return current_state, current_h
        elif best_neighbor_h > current_h:
            return current_state, current_h
        current_state, current_h = best_neighbor_state, best_neighbor_h


In [72]:
def n_queens(n: int, restarts: int = 0, side_way_moves: int = 0, depth: int | None = None, rd_seed: int | None = None):
    if depth is None:
        depth = n
    elif depth < 1:
        raise ValueError("depth must be at least 1")
    if rd_seed:
        random.seed(rd_seed)
    else:
        random.seed()

    initial_state = create_initiate_state(n)
    best_state, best_h = hill_climbing(initial_state, depth, side_way_moves)
    if best_h == 0:
        return best_state, best_h
    for restart in range(restarts):
        initial_state = create_initiate_state(n)
        current_best_state, current_best_h = hill_climbing(initial_state, depth, side_way_moves)
        if current_best_h < best_h:
            best_state, best_h = current_best_state, current_best_h
        if current_best_h == 0:
            break
    return best_state, best_h

In [73]:
n = 8
best_state, best_h = n_queens(n, rd_seed=42)
print(best_h)
print(get_matrix_view(best_state, n))

2
[[0 0 0 0 1 0 0 0]
 [1 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1]
 [0 0 1 0 0 0 0 0]
 [0 0 0 0 0 1 0 0]]


# Visualization

In [126]:
TILE_SIZE = 40
FONT_SIZE = 20
FONT_COLOR = (0, 0, 0)
BLACK_TILE_COLOR = (0, 0, 0)
WHITE_TILE_COLOR = (255, 255, 255)
QUEEN_COLOR = (176, 65, 65)
CIRCLE_RADIUS = TILE_SIZE / 2 - 2

In [127]:
def draw_tile(surface, position, color, circle=False, offset=(0, 0)):
    x_pos, y_pos = position
    x_offset, y_offset = offset
    x, y = x_pos + x_offset, y_pos + y_offset

    if circle:
        pygame.draw.circle(surface=surface, color=color,
                           center=(x * TILE_SIZE + TILE_SIZE / 2, y * TILE_SIZE + TILE_SIZE / 2), radius=CIRCLE_RADIUS)
    else:
        pygame.draw.rect(surface=surface, color=color,
                         rect=(x * TILE_SIZE, y * TILE_SIZE, TILE_SIZE, TILE_SIZE))


def draw_tiles(surface, positions, color, circle=False, display=True, offset=(0, 0)):
    for position in positions:
        draw_tile(surface, position, color, circle, offset)
    if display:
        pygame.display.flip()


def draw_text(position, font, screen, text, color):
    img_expanded = font.render(text, True, color)
    screen.blit(img_expanded, position)


def draw_chessboard(screen, n, offset=(0, 0), display=True):
    x_offset, y_offset = offset
    for x in range(n):
        for y in range(n):
            color = WHITE_TILE_COLOR
            if (x + y) % 2 == 0:
                color = BLACK_TILE_COLOR
            draw_tile(screen, (x + x_offset, y + y_offset), color)
    if display:
        pygame.display.flip()

In [139]:
def run_visualization(n: int, side_way_moves: int = 0, depth: int | None = None,
                      rd_seed: int | None = None):
    if depth is None:
        depth = n
    elif depth < 1:
        raise ValueError("depth must be at least 1")

    if rd_seed:
        random.seed(rd_seed)
    else:
        random.seed()

    pygame.init()
    screen = pygame.display.set_mode((2 * n * TILE_SIZE + TILE_SIZE, n * TILE_SIZE + 4 * FONT_SIZE))
    font = pygame.font.Font("roboto.ttf", FONT_SIZE)

    screen.fill("gray")
    pygame.display.flip()

    initial_state = create_initiate_state(n)
    initial_h = heuristic(initial_state)

    draw_chessboard(screen, n, display=False)
    draw_text(position=(TILE_SIZE / 2, n * TILE_SIZE + FONT_SIZE / 2), font=font, screen=screen, text="INITIAL",
              color=FONT_COLOR)
    draw_text(position=(TILE_SIZE / 2, n * TILE_SIZE + 2 * FONT_SIZE), font=font, screen=screen, text=f"h: {initial_h}",
              color=FONT_COLOR
              )
    draw_tiles(surface=screen, positions=initial_state, color=QUEEN_COLOR, circle=True, display=False)
    pygame.display.flip()

    best_state, best_h = hill_climbing(initial_state, depth, side_way_moves)

    draw_chessboard(screen, n, (n + 1, 0))
    draw_text(position=(TILE_SIZE * n + 1.5 * TILE_SIZE, n * TILE_SIZE + FONT_SIZE / 2), font=font, screen=screen,
              text="FINAL",
              color=FONT_COLOR)
    draw_text(position=(TILE_SIZE * n + 1.5 * TILE_SIZE, n * TILE_SIZE + 2 * FONT_SIZE), font=font, screen=screen,
              text=f"h: {best_h}",
              color=FONT_COLOR)
    draw_tiles(surface=screen, positions=best_state, color=QUEEN_COLOR, circle=True, display=False, offset=(n + 1, 0))
    pygame.display.flip()

    running = True

    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
    pygame.quit()

In [140]:
run_visualization(8, rd_seed=42)